In [1]:
# Giving the current working repository access.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Clearing sample_data extra directory.
!rm -r sample_data

In [3]:
# The directory structure for the working repository.
!ls drive/My\ Drive/touche-2022-prototyping

contrastive-learning-retrieval-comparison-pipeline.ipynb
dataset-prep-and-retrieval-diagnostic-analysis.ipynb
document_embeddings.pickle
indexes
initial-retrieval-metric-analysis.ipynb
levirank_bm25_voting_retrieval.ipynb
levirank_mon_duo_retrieval_module.ipynb
levirank-voting-and-average-recall-prototyping.ipynb
topic_embeddings.pickle
topics-task2-2022
topics-task2-51-100.xml
topics-task-2.xml
touche2020-task2-relevance-withbaseline.qrels
touche_complete_topics.csv
touche_ground_truth.csv
touche_results_2021.csv
touche-task2-51-100-relevance.qrels
touche-task2-passages-version-002-expanded-with-doc-t5-query.jsonl
touche_topics_query_expansion.csv


In [4]:
# Install pygaggle before transformers (first create the index using pyserini then install pygaggle)
!pip install pygaggle==0.0.2
!pip install transformers==4.17.0

     |████████████████████████████████| 43 kB 1.8 MB/s 
     |████████████████████████████████| 7.3 MB 4.8 MB/s 
     |████████████████████████████████| 60.4 MB 1.3 MB/s 
     |████████████████████████████████| 2.9 MB 41.1 MB/s 
     |████████████████████████████████| 60 kB 6.5 MB/s 
     |████████████████████████████████| 1.4 MB 50.8 MB/s 
     |████████████████████████████████| 1.1 MB 51.5 MB/s 
     |████████████████████████████████| 86 kB 6.3 MB/s 
     |████████████████████████████████| 1.1 MB 45.7 MB/s 
     |████████████████████████████████| 895 kB 50.1 MB/s 
     |████████████████████████████████| 462 kB 43.3 MB/s 
  Created wheel for pygaggle: filename=pygaggle-0.0.2-py3-none-any.whl size=43182 sha256=e17194c40d3ce934bd52d8097be7beb65b7455f95f4b688181f404e0e33cf774
  Stored in directory: /root/.cache/pip/wheels/fc/f9/b5/429f4a134742675c8de24a62d5d0ee4cea0e5e29267b2e7212
Successfully built pygaggle
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.0
    Un

In [5]:
# Making sure of the pygaggle's compatability with the correct transformers library.
import transformers
import pygaggle
import pandas as pd
from tqdm import tqdm
print(transformers.__version__)

4.17.0


In [6]:
# MonoT5 document re-ranking model execution.
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import MonoT5
reranker =  MonoT5()

In [9]:
# Getting the 2000 initial retrievals for each query.
relevant_bm25=pd.read_csv("/content/drive/MyDrive/touche-2022-prototyping/topics-task2-2022/monoT5-input-2000.csv")

In [10]:
# Printing out the dataframe output.
relevant_bm25.head()

,title_id,title,doc_id,content
0,2,Which is better a laptop or a desktop,clueweb12-0608wb-66-03868___1,Desktop or Laptop What the Tech HomeForumsCont...
1,2,Which is better a laptop or a desktop,clueweb12-0001wb-72-30848___8,Desktop replacements are usually larger and he...
2,2,Which is better a laptop or a desktop,clueweb12-0907wb-60-03588___2,There has been a turnaround in pricing trends ...
3,2,Which is better a laptop or a desktop,clueweb12-1804wb-90-15876___2,Therefore hard drives in a laptop usually have...
4,2,Which is better a laptop or a desktop,clueweb12-0601wb-28-19879___1,Do you Like Laptops or Desktops Better Mister ...


In [11]:
# Storing the order of columns in list form.
columns_order = relevant_bm25.columns

In [12]:
# Checking whether the below stated conditions are working correctly for
# MonoT5's input.
passages= relevant_bm25.loc[relevant_bm25["title"]=='Which is better a laptop or a desktop']
passages

,title_id,title,doc_id,content
0,2,Which is better a laptop or a desktop,clueweb12-0608wb-66-03868___1,Desktop or Laptop What the Tech HomeForumsCont...
1,2,Which is better a laptop or a desktop,clueweb12-0001wb-72-30848___8,Desktop replacements are usually larger and he...
2,2,Which is better a laptop or a desktop,clueweb12-0907wb-60-03588___2,There has been a turnaround in pricing trends ...
3,2,Which is better a laptop or a desktop,clueweb12-1804wb-90-15876___2,Therefore hard drives in a laptop usually have...
4,2,Which is better a laptop or a desktop,clueweb12-0601wb-28-19879___1,Do you Like Laptops or Desktops Better Mister ...
...,...,...,...,...
1995,2,Which is better a laptop or a desktop,clueweb12-0715wb-56-06863___2,There oughta be a better way says Alfred Poor ...
1996,2,Which is better a laptop or a desktop,clueweb12-0900tw-04-00469___21,Dell s affordable Inspiron family has always b...
1997,2,Which is better a laptop or a desktop,clueweb12-0901tw-12-09162___3,Even the cracker thin ASUS UL A A Three lbs is...
1998,2,Which is better a laptop or a desktop,clueweb12-1101wb-50-21469___6,Windows XP Operating System is user friendly a...


In [13]:
# Reranking the 2000 documents for each query as input.
new=[]
for title in tqdm(relevant_bm25["title"].unique()):
    passages= relevant_bm25.loc[relevant_bm25["title"]==title]
    texts = [ Text(p[1].content, {'docid': p[1].doc_id}, 0) for p in passages.iterrows()]
    query = Query(title)
    reranked = reranker.rerank(query, texts)
    for i in range(0, 2000):
        new.append({"doc_id":reranked[i].metadata["docid"], "Score":reranked[i].score, "title":title})

  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:195: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
100%|██████████| 50/50 [1:16:01<00:00, 91.22s/it]


In [14]:
# Creating the reranked document dataframe for monoT5.
df=pd.DataFrame(new)
df.head()

,doc_id,Score,title
0,clueweb12-0608wb-66-03868___1,-0.064696,Which is better a laptop or a desktop
1,clueweb12-0001wb-72-30848___8,-0.030760,Which is better a laptop or a desktop
2,clueweb12-0907wb-60-03588___2,-0.015391,Which is better a laptop or a desktop
3,clueweb12-1804wb-90-15876___2,-0.011190,Which is better a laptop or a desktop
4,clueweb12-0601wb-28-19879___1,-1.062416,Which is better a laptop or a desktop


In [15]:
# Adding the score column in the previous column order.
columns_order = ['title_id', 'title', 'doc_id', 'score', 'content']

In [16]:
# Preprocessing required for the join condition to work correctly below.
import re
def query_basic_preprocess(input_query):
    # input_query = input_query.lower()
    # Remove punctuation and numbers.
    input_query = re.sub(r"[^A-Za-z\s]+", " ", input_query)
    # Remove extra white spaces.
    input_query = re.sub(r"[\s]{2,}", " ", input_query)
    # Tokenize string.
    # word_tokens = word_tokenize(input_query)
    # Remove stop words.
    # word_tokens = [token for token in word_tokens if token not in stop_words]
    # word_tokens = [token for token in word_tokens if len(token) > 1]
    # Combining word tokens for results.
    # results = " ".join(word_tokens)
    output_query = input_query.strip()
    return output_query

In [17]:
# Making the reranked file in the same format as intial retrieval file
topics=pd.read_csv("/content/drive/MyDrive/touche-2022-prototyping/topics-task2-2022/topics-task2.csv")
topics['Title'] = topics['Title'].apply(lambda x: query_basic_preprocess(x))
df=df.join(topics[["Number","Title"]].set_index("Title"), on="title")
df=df.join(relevant_bm25[["doc_id","content"]].drop_duplicates(subset=['doc_id']).set_index("doc_id"), on="doc_id")
df=df.rename(columns={"Number":"title_id","Score":"score"})
df=df[columns_order]
df

,title_id,title,doc_id,score,content
0,2,Which is better a laptop or a desktop,clueweb12-0608wb-66-03868___1,-0.064696,Desktop or Laptop What the Tech HomeForumsCont...
1,2,Which is better a laptop or a desktop,clueweb12-0001wb-72-30848___8,-0.030760,Desktop replacements are usually larger and he...
2,2,Which is better a laptop or a desktop,clueweb12-0907wb-60-03588___2,-0.015391,There has been a turnaround in pricing trends ...
3,2,Which is better a laptop or a desktop,clueweb12-1804wb-90-15876___2,-0.011190,Therefore hard drives in a laptop usually have...
4,2,Which is better a laptop or a desktop,clueweb12-0601wb-28-19879___1,-1.062416,Do you Like Laptops or Desktops Better Mister ...
...,...,...,...,...,...
99995,100,Should I learn Python or R for data analysis,clueweb12-1414wb-70-17620___6,-11.514362,Increase the number of lines in the file by a ...
99996,100,Should I learn Python or R for data analysis,clueweb12-1500wb-41-29812___137,-13.086449,Changes to the library The wave module platfor...
99997,100,Should I learn Python or R for data analysis,clueweb12-1500wb-41-29812___265,-11.890977,For long integers a minus sign is used as befo...
99998,100,Should I learn Python or R for data analysis,clueweb12-1613wb-61-18874___9,-9.384537,Requires additional space Radix sort needs add...


In [21]:
# Storing the final dataframe for re-ranking document input for DuoT5.
df.to_csv("/content/drive/MyDrive/touche-2022-prototyping/topics-task2-2022/monoT5-voting-reranked.csv")

In [19]:
# DuoT5 document document reranking process.
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import DuoT5
reranker =  DuoT5()

In [22]:
# Loading the MonoT5 re-ranked documents into a dataframe.
mono=pd.read_csv("/content/drive/MyDrive/touche-2022-prototyping/topics-task2-2022/monoT5-voting-reranked.csv")
mono.head()

,Unnamed: 0,title_id,title,doc_id,score,content
0,0,2,Which is better a laptop or a desktop,clueweb12-0608wb-66-03868___1,-0.064696,Desktop or Laptop What the Tech HomeForumsCont...
1,1,2,Which is better a laptop or a desktop,clueweb12-0001wb-72-30848___8,-0.030760,Desktop replacements are usually larger and he...
2,2,2,Which is better a laptop or a desktop,clueweb12-0907wb-60-03588___2,-0.015391,There has been a turnaround in pricing trends ...
3,3,2,Which is better a laptop or a desktop,clueweb12-1804wb-90-15876___2,-0.011190,Therefore hard drives in a laptop usually have...
4,4,2,Which is better a laptop or a desktop,clueweb12-0601wb-28-19879___1,-1.062416,Do you Like Laptops or Desktops Better Mister ...


In [23]:
# Reranking the 2000 documents for each query as input.
new=[]
for title in tqdm(mono["title"].unique()):
  passages= mono.loc[mono["title"]==title].sort_values(by="score",ascending=False)
  texts = [ Text(p[1].content, {'docid': p[1].doc_id}, p[1].score) for p in passages.iloc[0:50,:].iterrows()]
  query = Query(title)
  reranked = reranker.rerank(query, texts)
  for i in range(0, 50):
    new.append({"doc_id":reranked[i].metadata["docid"], "Score":reranked[i].score, "title":title})
  for i in range(50, 1000):
    new.append({"doc_id":passages.iloc[i]["doc_id"], "Score":passages.iloc[i]["score"], "title":title})

  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:195: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
100%|██████████| 50/50 [2:26:53<00:00, 176.27s/it]


In [24]:
# Adding the score column in the previous column order.
columns_order=mono.columns
columns_order=['title_id', 'title', 'doc_id', 'score', 'content']
print(columns_order)

['title_id', 'title', 'doc_id', 'score', 'content']


In [25]:
# Redeclaring this function in case runtime get disconnected.
# Preprocessing required for the join condition to work correctly below.
import re
def query_basic_preprocess(input_query):
    # input_query = input_query.lower()
    # Remove punctuation and numbers.
    input_query = re.sub(r"[^A-Za-z\s]+", " ", input_query)
    # Remove extra white spaces.
    input_query = re.sub(r"[\s]{2,}", " ", input_query)
    # Tokenize string.
    # word_tokens = word_tokenize(input_query)
    # Remove stop words.
    # word_tokens = [token for token in word_tokens if token not in stop_words]
    # word_tokens = [token for token in word_tokens if len(token) > 1]
    # Combining word tokens for results.
    # results = " ".join(word_tokens)
    output_query = input_query.strip()
    return output_query

In [26]:
# Making the reranked file in the same format as intial retrieval file.
df=pd.DataFrame(new)
topics=pd.read_csv("/content/drive/MyDrive/touche-2022-prototyping/topics-task2-2022/topics-task2.csv")
topics['Title'] = topics['Title'].apply(lambda x: query_basic_preprocess(x))
df=df.join(topics[["Number","Title"]].set_index("Title"), on="title")
df=df.join(mono[["doc_id","content"]].drop_duplicates(subset=['doc_id']).set_index("doc_id"), on="doc_id")
df=df.rename(columns={"Number":"title_id","Score":"score","Title":"title"})
df=df[columns_order]
df

,title_id,title,doc_id,score,content
0,2,Which is better a laptop or a desktop,clueweb12-1804wb-90-15876___2,78.821897,Therefore hard drives in a laptop usually have...
1,2,Which is better a laptop or a desktop,clueweb12-0301wb-84-10147___2,85.427993,Many times consumers may opt to have both but ...
2,2,Which is better a laptop or a desktop,clueweb12-0907wb-60-03588___2,79.229755,There has been a turnaround in pricing trends ...
3,2,Which is better a laptop or a desktop,clueweb12-0013wb-19-15392___6,72.569362,Keep in mind a laptop is one unit If you break...
4,2,Which is better a laptop or a desktop,clueweb12-0608wb-66-03868___2,71.743426,Laptop computers are portable have the virtual...
...,...,...,...,...,...
49995,100,Should I learn Python or R for data analysis,clueweb12-0204wb-38-15941___33,-11.528502,cells which cell setval ord char ord Attribute...
49996,100,Should I learn Python or R for data analysis,clueweb12-0909wb-33-12181___40,-11.529103,Lisp has been around a long time and has watch...
49997,100,Should I learn Python or R for data analysis,clueweb12-0311wb-17-16828___2,-11.530167,Its teaching should therefore be regarded as a...
49998,100,Should I learn Python or R for data analysis,clueweb12-0001wb-56-00448___192,-11.530614,After that we plot these N estimates on the y ...


In [27]:
# Storing the final dataframe for re-ranking document input for DuoT5.
df.to_csv("/content/drive/MyDrive/touche-2022-prototyping/topics-task2-2022/duoT5-voting-reranked.csv",index=False)